In [38]:
wires: dict[str, dict[int, int]] = {}
gates = []
gate_mode = False
with open('input.txt') as f:
    for i, line in enumerate(f):
        line = line.strip()
        if line == '':
            gate_mode = True
            continue

        if gate_mode:
            io = line.split('->')
            obj = io[0].strip().split(' ')
            obj = {'type': obj[1], 'left': obj[0], 'right': obj[2], 'output': io[1].strip()}
            gates.append(obj)
        else:
            w_name = line[0:3]
            w_value = bool(int(line[5]))
            wires[w_name] = w_value


print(wires)
print(gates)

{'x00': True, 'x01': False, 'x02': True, 'x03': True, 'x04': False, 'x05': False, 'x06': True, 'x07': True, 'x08': False, 'x09': True, 'x10': True, 'x11': True, 'x12': True, 'x13': False, 'x14': True, 'x15': True, 'x16': True, 'x17': True, 'x18': True, 'x19': True, 'x20': False, 'x21': True, 'x22': False, 'x23': True, 'x24': False, 'x25': True, 'x26': True, 'x27': True, 'x28': True, 'x29': False, 'x30': False, 'x31': True, 'x32': False, 'x33': True, 'x34': True, 'x35': False, 'x36': False, 'x37': True, 'x38': False, 'x39': True, 'x40': True, 'x41': True, 'x42': True, 'x43': False, 'x44': True, 'y00': True, 'y01': False, 'y02': False, 'y03': True, 'y04': True, 'y05': False, 'y06': False, 'y07': False, 'y08': False, 'y09': False, 'y10': False, 'y11': True, 'y12': False, 'y13': True, 'y14': False, 'y15': True, 'y16': True, 'y17': True, 'y18': True, 'y19': False, 'y20': False, 'y21': True, 'y22': True, 'y23': True, 'y24': True, 'y25': False, 'y26': False, 'y27': True, 'y28': True, 'y29': T

This looks like a clear Graph Theory problem. We can define each gate as a node and each wire as an edge. We may have to invent nodes for the root.

For each wire, $w_i$ there are two nodes $a_i$ and $b_i$
In order to find the value of a given node $n_i$, we must detect if parent nodes are already calculated. If not, we must calculate them first, and so on.
So define a recursive function $f(n_i)$ that returns the value of node $n_i$.

In [39]:
class Node():
    def __init__(self, type: str, name: str, left: 'Node', right: 'Node', graph):
        self.type = type
        self.name = name
        self.value: bool = None
        self.graph = graph
        self.left = left
        self.right = right

    def calculate(self):
        if self.value is not None:
            return self.value

        self.connect()

        if self.type == 'AND':
            return self.and_gate()
        elif self.type == 'OR':
            return self.or_gate()
        elif self.type == 'XOR':
            return self.xor_gate()

    def and_gate(self):
        self.value = self.left.calculate() & self.right.calculate()
        return self.value
    
    def or_gate(self):
        self.value = self.left.calculate() | self.right.calculate()
        return self.value
    
    def xor_gate(self):
        self.value = self.left.calculate() ^ self.right.calculate()
        return self.value
    
    def connect(self):
        if isinstance(self.left, str) or isinstance(self.right, str):
            self.left = self.graph[self.left]
            self.right = self.graph[self.right]

    def __str__(self):
        return f'{self.left} {self.type} {self.right} -> {self.name}'
    
    def __repr__(self):
        return f'{self.left} {self.type} {self.right} -> {self.name}'

In [40]:
graph = {}
for gate in gates:
    graph[gate['output']] = Node(gate['type'], gate['output'], gate['left'], gate['right'], graph)

for wire in wires.keys():
    graph[wire] = Node(wire, wire, None, None, graph)
    graph[wire].value = wires[wire]

z = []
for key, value in graph.items():
    if key.startswith('z'):
        z.append((key, int(value.calculate())))

# order by tuple index 0
z.sort(key=lambda x: x[0])
z = [x[1] for x in z]
z

# convert binary array to int
res = sum(x * 2**i for i, x in enumerate(z))
res

56620966442854